In [1]:
from Bio import Entrez
Entrez.email = "artem_vasilev_01@list.ru"

In [2]:
handle = Entrez.esearch(db="pubmed", term="nxf1")

In [3]:
record = Entrez.read(handle)
print(record)

{'Count': '284', 'RetMax': '20', 'RetStart': '0', 'IdList': ['39270556', '39108207', '38799568', '38768348', '38626369', '38593806', '38165804', '37909783', '37679383', '37628773', '37583452', '37540751', '37143720', '37069509', '36920996', '36857431', '36658633', '36040180', '35751040', '35589130'], 'TranslationSet': [], 'QueryTranslation': '"nxf1"[All Fields]'}


In [8]:
mshandle = Entrez.efetch(
db="pubmed", id=record["IdList"][0], rettype="abstract", retmode="text"
)
print(mshandle.read())

1. Biochem Biophys Res Commun. 2024 Aug 28;735:150608. doi: 
10.1016/j.bbrc.2024.150608. Online ahead of print.

Interaction of CDK12 with NXF1 is a new node for the linking mechanism between 
transcription and transportation of mRNA.

Ebara S(1), Yoshida M(2), Yamakawa H(1), Shimokawa K(2), Inoue Y(3), Tauchi 
H(4), Morishita D(5).

Author information:
(1)Department of Biological Sciences, Faculty of Science, Ibaraki University, 
Bunkyo 2-1-1, Mito, Ibaraki, 310-8512, Japan; Chordia Therapeutics Inc., 
Kanagawa, 251-0012, Japan.
(2)Chordia Therapeutics Inc., Kanagawa, 251-0012, Japan.
(3)Department of Cell Signaling, Graduate School of Pharmaceutical Sciences, 
Nagoya City University, Nagoya, 467-8603, Japan.
(4)Department of Biological Sciences, Faculty of Science, Ibaraki University, 
Bunkyo 2-1-1, Mito, Ibaraki, 310-8512, Japan. Electronic address: 
hiroshi.tauchi.sci@vc.ibaraki.ac.jp.
(5)Chordia Therapeutics Inc., Kanagawa, 251-0012, Japan. Electronic address: 
daisuke.morishita@c

---

In [9]:
handle = Entrez.esearch(
db="nucleotide", term="nxf1 AND Drosophila melanogaster[orgn]"
) # db="genbank" Error!!
record = Entrez.read(handle)
Entrez.efetch(db="nucleotide", id=record["IdList"])
handle = Entrez.esearch(db="taxonomy", term="Drosophila melanogaster")
record = Entrez.read(handle)
print(record)
print(record["IdList"])

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['7227'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'drosophila melanogaster[All Names]', 'Field': 'All Names', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'drosophila melanogaster[All Names]'}
['7227']


---

In [11]:
handle = Entrez.esearch(db="protein", term="nxf1 AND Drosophila melanogaster[orgn]")
record = Entrez.read(handle)
for rec in record["IdList"]:
    temphandle = Entrez.read(Entrez.esummary(db="protein", id=rec, retmode="text"))
    print(
        temphandle[0]["Id"]
        + "\t"
        + temphandle[0]["Caption"]
        + "\t"
        + str(int(temphandle[0]["Length"]))
    ) # +"\n")

20978541	Q9VV73	841
13124798	Q24168	618
1376014748	A0A0B4K7J2	2718
20455240	Q9VIW3	596
74948793	Q9VYX1	101
74948518	Q9VTL1	395
20978539	Q9U1H9	672
18203549	Q9V3H8	133
2282536378	Q9U3V9	2079
1624698291	NP_001356942	536
1624698289	NP_001356941	536
17933682	NP_524660	672
1622462254	QCD25237	536
1622462253	QCD25236	536
22832060	AAF47959	672
1727108961	6IHJ_D	135
1727108960	6IHJ_B	135
1727108959	6IHJ_C	191
1727108958	6IHJ_A	191
1700031391	6OPF_F	98


---

In [14]:
Entrez.efetch(db="protein", id=record["IdList"], retmode="text", rettype="fasta").read()

with open("Drmel_nxf1.fa", "w") as ouf:
    for rec in record["IdList"]:
        lne = Entrez.efetch(
            db="protein", id=rec, retmode="text", rettype="fasta"
        ).read()
        ouf.write(lne + "\n")

In [ ]:
with open("Drmel_nxf1.fa", "r") as fastaf:
    snippet = [next(fastaf) for x in range(5)]
    print(snippet)

---

In [43]:
handle = Entrez.esearch(db="nucleotide", term="nxf1 AND Drosophila melanogaster[orgn]") # db="genbank" Error!!
record = Entrez.read(handle)

In [44]:
record["IdList"]

['1624698290', '1624698288', '669632474', '281360686', '667695275', '14456154', '14456089']

In [45]:
for rec in record["IdList"]:
    temphandle = Entrez.read(Entrez.esummary(db="nucleotide", id=rec, retmode="text"))
    print(
        temphandle[0]["Id"]
        + "\t"
        + temphandle[0]["Caption"]
        + "\t"
        + str(int(temphandle[0]["Length"]))
    ) # +"\n")

1624698290	NM_001369982	2147
1624698288	NM_001369981	1910
669632474	NC_004354	23542271
281360686	NM_079921	3280
667695275	AE014298	23542271
14456154	AJ318090	2522
14456089	AJ312282	2526


AE - NCBI - Genome project
AJ - EBI - Direct submissions
NM - curated RefSeq

In [47]:
with open("Dm_nxf1_transcript_var_A.fa", "w") as outfile:
    lne = Entrez.efetch(
        db="nucleotide", id=record["IdList"][3], retmode="text", rettype="fasta"
    ).read()
    outfile.write(lne + "\n")

https://www.alliancegenome.org/gene/FB:FBgn0003321

Sequence Details (Mode: cDNA)

---

1. Найти референсные последовательности для sbr - https://www.alliancegenome.org/gene/FB:FBgn0003321 (Sequence Details (Mode: cDNA)):
   - найти координаты гены на X хромосоме
   - полная последовательность гена
   - сплайсированный вариант
   - консервативная кассета
2. Написать срипт, который:
   - ищет ген sbr / nxf1 в базе данных NCBI для семейства Drosophilidae
   - выбирает из найденных только адекватные варианты и формирует из них БД для бласта
3. Сделать бласт вариантов из 1-го пункта на варианты из второго пункта

In [41]:
from functions import plain_to_fasta

In [42]:
path_to_plain_dir = "/home/artemvaska/Master_degree/Diploma/References/"

In [ ]:
plain_to_fasta(path_to_plain_dir + "sbr_RA_gene_plain.fa", fasta_line_length=80, uppercase=True)

In [45]:
plain_to_fasta(path_to_plain_dir + "sbr_RA_CDS_plain.fa", fasta_line_length=80, uppercase=True)

In [52]:
SBR_CONSERVATIVE_CASSETTE = 'ATACCATCTTTGGCCCACCTTGTAGTGCTTCGCAATCTGTCCATCTTGGAACTCGTTTTAAAGAACAATCCCTGCCGTTCCCGCTACAAGGATTCCCAGCAGTTTATCAGGTATACTATGGTGTGATTCGTTGATCTTCTAACTTTTCTAGTGGATGCTGCCGCATATTCGACGTGGGATTTGTCGCGTCTGGAGTTACAGAGGAATTAGGATTACCACATTTCCATTGTTCTGTTGTCTCCATTCCTTTTCTTTGAAGCAAATGCGCTGCAGAATTGGGGACCAAGGTCAGGTGTTGCCTTCGTCCAAAGTTTAAAGCAGCTGAGCAGGCAAAGGCATTTGGGAAATATTTTCCTTATCAGCTGTCTTAAAGCCAGGACAGATCGTATGCAGCGGAATGGATTTTGGGCAGAGCAAAAAAAAAAAAAAGAAGAAAGAAAAAAAAAGATTTGAAGTTAAAAAAAAAAAAGAAAAAAAAAGAAGAAAAAAAAAGAAAAAAAAGTACAAGCAAAAAAGGCCCAAGCCAATTGCAGTAAACTTTCGACGATGTGGCAACGCTAAACTAAGAACGCAACTGTACTTTTGACACACACACACCGGCACCGTTCGTACTTCCAATTGAGCAGGCGGACGTGCAGCAGTCCCAAAGCGAATCACATTAACGCATTAACATCGACAAATATAAATTATTTTACGAATAACAAATAATCCACCGAACCCACCCACCGCGCACCCCGCTACAACACACACACGGGCGCACGCATGTAGCCACTCCACGCCAGCGCACGCACGCATGCGCATGCGCATATGCAGTTGTGTACGTGCCACATCTAATGCTGCGTGCCCTGTGCGCGCCGTTGCGCCTTGCATTGTCGTGTCGCCGCTTGTTGATTTTGCGGTGGTTGCCAAGTAGTGGAGGCAGAGTGGAGTGCATAGGCTCACCATTCGGGTGCGCAGCGTCCAGCAGGAAGCAGAAACGACTACCAAGACGAAGGACACACCACTGACTGACTCACGTGCTAACGGACATTCTGCTGGCGCCAGCGGCCGGCGCGTTGTTTTGCCAGTCGATTACAATTTACAATTAGTATTTTCGGCTTTGTTGGGCGCATGAGCCGCCTGCGGATGCGACGTGCTTGAACATGACGAGACGAACAGAAGTTACACAGCAGTCAGCTTTGAAAGCCACTTTTTGCTTGTGGCCACTGAAAGAAAAAACAAAAAAAAAAAAATGAAAAAAAGGAAAAGAAAAAAGTGTTGATAAAAAAAAAAAAACAAAAAATACAAAAGAGAAAATACCACAAAAAAGATGCCCCGGCACAATGCACTGCTCAATTGTACTTTTCCGCTGTACTTTTCACATCGAACTATTGAAGGCACAACAATGGCCAAGCACCTAGTGCCTCTGCCTCTTCAGCTGATCAAAGCTCCGCCAATTTGTGTGTGAGCCGGTCGAGTAACGGTGAAGCTATATCAAAGATACAAACATATATACTCCGCGTGAGAAGAAGAAGGAGTCACATCTGGCCACTGACTAATCTGCAATTCCTGCAGCGCATACATCCTAAACCTCCCCCCCCCCCCACCAACGCACAAATTTAAAAAAAAAACCCCCATTTAATTATTATTTCTCGCTATTTAACTCGCATATCTATCATCGCCTCCCTGTGCTAATGACAACAAAAATACGGCCGAATGTCCACTATTTACAGCGAAGTACGTCGCAAGTTTCCCAAACTGGTTAAGTTG'

In [55]:
SBR_CASSETTE_INTRON = 'GTATACTATGGTGTGATTCGTTGATCTTCTAACTTTTCTAGTGGATGCTGCCGCATATTCGACGTGGGATTTGTCGCGTCTGGAGTTACAGAGGAATTAGGATTACCACATTTCCATTGTTCTGTTGTCTCCATTCCTTTTCTTTGAAGCAAATGCGCTGCAGAATTGGGGACCAAGGTCAGGTGTTGCCTTCGTCCAAAGTTTAAAGCAGCTGAGCAGGCAAAGGCATTTGGGAAATATTTTCCTTATCAGCTGTCTTAAAGCCAGGACAGATCGTATGCAGCGGAATGGATTTTGGGCAGAGCAAAAAAAAAAAAAAGAAGAAAGAAAAAAAAAGATTTGAAGTTAAAAAAAAAAAAGAAAAAAAAAGAAGAAAAAAAAAGAAAAAAAAGTACAAGCAAAAAAGGCCCAAGCCAATTGCAGTAAACTTTCGACGATGTGGCAACGCTAAACTAAGAACGCAACTGTACTTTTGACACACACACACCGGCACCGTTCGTACTTCCAATTGAGCAGGCGGACGTGCAGCAGTCCCAAAGCGAATCACATTAACGCATTAACATCGACAAATATAAATTATTTTACGAATAACAAATAATCCACCGAACCCACCCACCGCGCACCCCGCTACAACACACACACGGGCGCACGCATGTAGCCACTCCACGCCAGCGCACGCACGCATGCGCATGCGCATATGCAGTTGTGTACGTGCCACATCTAATGCTGCGTGCCCTGTGCGCGCCGTTGCGCCTTGCATTGTCGTGTCGCCGCTTGTTGATTTTGCGGTGGTTGCCAAGTAGTGGAGGCAGAGTGGAGTGCATAGGCTCACCATTCGGGTGCGCAGCGTCCAGCAGGAAGCAGAAACGACTACCAAGACGAAGGACACACCACTGACTGACTCACGTGCTAACGGACATTCTGCTGGCGCCAGCGGCCGGCGCGTTGTTTTGCCAGTCGATTACAATTTACAATTAGTATTTTCGGCTTTGTTGGGCGCATGAGCCGCCTGCGGATGCGACGTGCTTGAACATGACGAGACGAACAGAAGTTACACAGCAGTCAGCTTTGAAAGCCACTTTTTGCTTGTGGCCACTGAAAGAAAAAACAAAAAAAAAAAAATGAAAAAAAGGAAAAGAAAAAAGTGTTGATAAAAAAAAAAAAACAAAAAATACAAAAGAGAAAATACCACAAAAAAGATGCCCCGGCACAATGCACTGCTCAATTGTACTTTTCCGCTGTACTTTTCACATCGAACTATTGAAGGCACAACAATGGCCAAGCACCTAGTGCCTCTGCCTCTTCAGCTGATCAAAGCTCCGCCAATTTGTGTGTGAGCCGGTCGAGTAACGGTGAAGCTATATCAAAGATACAAACATATATACTCCGCGTGAGAAGAAGAAGGAGTCACATCTGGCCACTGACTAATCTGCAATTCCTGCAGCGCATACATCCTAAACCTCCCCCCCCCCCCACCAACGCACAAATTTAAAAAAAAAACCCCCATTTAATTATTATTTCTCGCTATTTAACTCGCATATCTATCATCGCCTCCCTGTGCTAATGACAACAAAAATACGGCCGAATGTCCACTATTTACAG'

In [71]:
def read_fasta(file: str) -> dict:
    seqs = {}
    seq = []
    with open(file) as infile:
        for line in infile:
            line = line.strip()
            if line.startswith(">"):
                header = line[1:]
            else:
                seq.append(line)
    seqs[header] = "".join(seq)
    return seqs

In [75]:
sbr_gene = read_fasta("../References/sbr_RA_CDS.fa")

In [79]:
sbr_gene['sbr-RA-cds']

'ATGCCCAAACGCGGCGGTGGCAGTAGCCAGCGGTACAACAACAACGTTGGAAATGGCGGCGGACGTTACAACGCTCCCGAGGATTTCGATGATTTTGGTCATGTGGAGGATCGCCAGCGACGCAAGGATCGAAACAAGCGGCGCGTCAGCTTTAAGCCCTCCCAATGTCTACATAACAAAAAGGACATCAAGCTGCGACCCGAAGATTTGCGTCGATGGGACGAGGATGATGACATGAGCGACATGACCACGGCCGTTAAGGATAGACCCACCTCCCGACGTCGGGGATCGCCCATTCCGCGCGGCAAGTTCGGCAAACTGATGCCCAACAGCTTTGGCTGGTACCAAGTCACGTTACAAAACGCCCAGATATACGAAAAGGAAACACTCTTGAGTGCTCTATTGGCAGCGATGTCGCCACATGTCTTTATTCCTCAATATTGGCGAGTGGAGCGAAACTGCGTAATCTTCTTTACGGACGACTACGAGGCAGCCGAACGCATTCAACATCTGGGCAAGAATGGCCATCTTCCAGATGGCTATCGTCTGATGCCACGAGTACGCAGCGGTATACCACTAGTGGCCATCGACGATGCCTTCAAGGAGAAGATGAAGGTCACAATGGCCAAGCGTTACAATATTCAAACCAAGGCGCTGGATCTTTCCCGTTTTCATGCAGATCCGGATCTTAAGCAAGTTTTCTGCCCACTCTTTCGTCAGAATGTGATGGGCGCTGCCATTGACATTATGTGCGACAATATACCCGATTTGGAGGCACTTAACCTGAATGACAACTCCATTAGCAGCATGGAGGCGTTTAAGGGTGTGGAGAAACGCTTACCGAACCTCAAGATTCTCTATTTGGGGGATAACAAGATACCATCTTTGGCCCACCTTGTAGTGCTTCGCAATCTGTCCATCTTGGAACTCGTTTTAAAGAACAATCCCTGCCGTTCCCGCTACAAGGATTCCCAGCAGTTTATCAGCGAAGTACGTCGC

https://biopython.org/docs/dev/Tutorial/chapter_blast.html

In [1]:
from Bio import Blast

In [2]:
Blast.tool

'biopython'

In [3]:
Blast.email = "artem_vasilev_01@list.ru"

In [4]:
fasta_string = open("../References/sbr_RA_gene.fa").read()

In [ ]:
"all [filter] NOT(environmental samples[organism] OR metagenomes[orgn]) AND txid3702[ORGN] AND txid9606[ORGN]"

In [67]:
result_stream = Blast.qblast(program="blastn",
                             database="wgs",
                             sequence=fasta_string,
                             entrez_query="Drosophilidae AND NOT Drosophila melanogaster",
                             megablast=True)

Drosophilidae (taxid:7214)

Drosophila melanogaster (taxid:7227)

https://biochem.slu.edu/bchm628/handouts/2013/Entrez_boolian_searches.pdf

In [71]:
with open("sbr_RA_gene_blast.xml", "wb") as out_stream:
    out_stream.write(result_stream.read())
result_stream.close()

result_stream = open("sbr_RA_gene_blast.xml", "rb")

blast_record = Blast.read(result_stream)

ValueError: premature end of XML file (after reading -1 bytes)

In [77]:
blast_record

<Bio.Blast.Record query.id='Query_6742799'; no hits>

In [78]:
print(blast_record)

Program: BLASTN 2.16.0+
     db: wgs
  Query: Query_6742799 (length=14341)
         sbr-RA-gene
   Hits: No hits found


In [79]:
from Bio.Blast.Applications import NcbiblastnCommandline

/home/artemvaska/mamba/envs/diploma/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [ ]:
blastn_cline = NcbiblastnCommandline(query = "search.fasta", db = "alun", 
outfmt = 5, out = "results.xml")

In [ ]:
stdout, stderr = blastn_cline()